<a href="https://colab.research.google.com/github/nechebarrena/HOML/blob/main/HOML_cap_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CAPITULO 9 - HANDS ON MACHINE LEARNING**
## **Aprendizaje no supervisado**

Aunque buena parte de la bibliografia se enfoca en los problemas de aprendizaje supervisado, estos en realidad son minoritarios frente a los problemas donde debemos utilizar tecnicas de aprendizaje no supervisado. En general es mas facil pensar que para un conjunto de instancias $\mathbf{X}$ no vamos a tener un conjunto de etiquetas $\mathbf{Y}$ que le correspondan.

Algunas tecnicas de aprendizaje no supervisado que podemos utilizar en general son:

1. Reduccion de dimensionalidad; Tal como vimos en el capitulo $8$ la reduccion de dimensionalidad nos permite trabajar con un numero reducido de las features originales. 

2. Clustering; El objetivo de esta tecnica es agrupar instancias segun alguna metrica de similaridad. La idea justamente es armar clusters lo mas disjuntos posibles.

3. Deteccion de anormalidades; Sirve para detectar justamente cuales son los datos "normales" de un determinado tipo e identificar los que estan fuera de este refimen de normalidad.

4. Estimacion de densidad; La idea es poder estimar la funcion densidad de probabilidad (PDF) de una muestra de instancias. 

Primero vamos a ver como realizar problemas de clustering y luego vamos a ver algo en menor profundidad de los otros.

## **Clustering**

En general los metodos de clustering se pueden usar para un monton de propositos distintos. Pueden ser usados para reducir la dimension de los datos, para asignar etiquetas, para asignar etiquetas cuando ya contamos con algunas de ellas, etc etc.. 

Pero hay una pregunta que parece previa y es, ¿que es un cluster?

La realidad es que no hay una definicion unica de lo que significa un cluster y segun el metodo que se use para resolver el problema se va a estar usando una definicion u otra, sin embargo intuitivamente es facil de pensar como un conjunto de instancias que comparten lo suficiente como para agruparlas entre si y son lo suficientemente distintas de otras como para separarlas de estas.

El libro se propone explicar el funcionamiento de dos de los metodos principales de clustering, DBSCAN y K-means.